# Lesson 2 : LangGraph Components

In [1]:
%pip install langgraph


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
_ = load_dotenv()

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

In [3]:
tool = TavilySearchResults(max_results=4) #increased number of results
print(type(tool))
print(tool.name)

<class 'langchain_community.tools.tavily_search.tool.TavilySearchResults'>
tavily_search_results_json


> If you are not familiar with python typing annotation, you can refer to the [python documents](https://docs.python.org/3/library/typing.html).

In [4]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

> Note: in `take_action` below, some logic was added to cover the case that the LLM returned a non-existent tool name. Even with function calling, LLMs can still occasionally hallucinate. Note that all that is done is instructing the LLM to try again! An advantage of an agentic organization.

In [5]:
class Agent:

    def __init__(self, model, tools, system=""):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_openai)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm",
            self.exists_action,
            {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile()
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState):
        result = state['messages'][-1]
        return len(result.tool_calls) > 0

    def call_openai(self, state: AgentState):
        messages = state['messages']
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {'messages': [message]}

    def take_action(self, state: AgentState):
        tool_calls = state['messages'][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            if not t['name'] in self.tools:      # check for bad tool name from LLM
                print("\n ....bad tool name....")
                result = "bad tool name, retry"  # instruct LLM to retry if bad
            else:
                result = self.tools[t['name']].invoke(t['args'])
            results.append(ToolMessage(tool_call_id=t['id'], name=t['name'], content=str(result)))
        print("Back to the model!")
        return {'messages': results}

In [9]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""

model = ChatOpenAI(model="gpt-3.5-turbo")  #reduce inference cost
abot = Agent(model, [tool], system=prompt)

In [7]:
# !python3 -m pip install -U --no-cache-dir  \
#             --config-settings="--global-option=build_ext" \
#             --config-settings="--global-option=-I$(brew --prefix graphviz)/include/" \
#             --config-settings="--global-option=-L$(brew --prefix graphviz)/lib/" \
#             pygraphviz

In [8]:
# !pip3 install pygraphviz

In [10]:
# !sudo apt-get install graphviz graphviz-dev

In [11]:
# from IPython.display import Image

# Image(abot.graph.get_graph().draw_png())

In [12]:
messages = [HumanMessage(content="What is the weather in sf?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_YV7eTtegZomgmDXRWqeR1sGg'}
Back to the model!


In [13]:
result

{'messages': [HumanMessage(content='What is the weather in sf?'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_YV7eTtegZomgmDXRWqeR1sGg', 'function': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 153, 'total_tokens': 174}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b8159dc1-3ba3-4314-be45-d2900ae57a8a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_YV7eTtegZomgmDXRWqeR1sGg'}]),
  ToolMessage(content='[{\'url\': \'https://world-weather.info/forecast/usa/san_francisco/june-2024/\', \'content\': \'Extended weather forecast in San Francisco. Hourly Week 10 days 14 days 30 days Year. Detailed ⚡ San Francisco Weather Forecast for June 2024 - day/night 🌡️ temperatures, precipitations -

In [14]:
result['messages'][-1].content

'The current weather in San Francisco is partly cloudy with a temperature of 57.0°F (13.9°C). The wind is blowing at 3.6 kph from the north. The humidity is at 83%, and the visibility is 9.0 miles.'

In [15]:
messages = [HumanMessage(content="What is the weather in SF and LA?")]
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in San Francisco'}, 'id': 'call_OiO62VhQ999PXQ9oiJCgtWhh'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'weather in Los Angeles'}, 'id': 'call_wVWHweICHe2EKRqtWnzn9d8q'}
Back to the model!


In [16]:
result['messages'][-1].content

'The current weather in San Francisco is partly cloudy with a temperature of 57.0°F. The wind speed is 3.6 km/h coming from the north. The humidity is at 83% with a visibility of 9.0 miles.\n\nIn Los Angeles, the current weather is also partly cloudy with a temperature of 66.0°F. The wind speed is 3.6 km/h coming from the north. The humidity is at 75% with a visibility of 9.0 miles.'

In [17]:
# Note, the query was modified to produce more consistent results. 
# Results may vary per run and over time as search information and models change.

query = "Who won the super bowl in 2024? In what state is the winning team headquarters located? \
What is the GDP of that state? Answer each question." 
messages = [HumanMessage(content=query)]

model = ChatOpenAI(model="gpt-4o")  # requires more advanced model
abot = Agent(model, [tool], system=prompt)
result = abot.graph.invoke({"messages": messages})

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'Super Bowl 2024 winner'}, 'id': 'call_JgaOn6vrPIdeKzuENnBn1uAl'}
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'GDP of the state of the 2024 Super Bowl winning team headquarters'}, 'id': 'call_RRy4kJbYHHll0Hi7KL1iBr43'}
Back to the model!
Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current GDP of Missouri 2024'}, 'id': 'call_aBrojKpYAQmyrzZvKw13nJ8u'}
Back to the model!


In [18]:
print(result['messages'][-1].content)

### What is the GDP of that state?
The most recent data for Missouri's GDP can be found on the Bureau of Economic Analysis website. As of the fourth quarter of 2023, Missouri's GDP growth was recorded, but the exact GDP figure for 2024 is not explicitly stated in the search results. For precise and up-to-date information, you may refer directly to the [Bureau of Economic Analysis (BEA) website](https://www.bea.gov/data/gdp/gdp-state).

In summary:
1. **Super Bowl 2024 Winner**: Kansas City Chiefs
2. **Winning Team Headquarters**: Missouri
3. **GDP of Missouri**: For the most accurate and current GDP figure, consult the BEA's latest state data releases.
